# CODE-DE/EO-Lab tutorial

<div style="text-align: right"><i> Beginner </i></div>

***
<center><h1> Prozessierung einer Vegetations-Zeitreihe mit FORCE DataCube Collection Daten vom Jupyter-Lab</h1></center>

***
**General Note 1**: Ausführung der Zellen durch pressen des <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button vom top MENU (oder `Shift` + `Enter`).<br>
<br>
**General Note 2**: Falls der Kern nich mehr arbeitet, im the top MENU, klicke <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Dann, im top MENU, clicke  "Run" aund wähle "Run All Above Selected Cell".<br>

**General Note 3**: Schauen Sie sich im [**FORUM**](https://forum.code-de.org/de/) um oder Kontaktieren Sie den Support ! <br>

***

In diesem Tutorial möchten wir die [FORCE Datacube Collection](https://code-de.org/en/portfolio/?id=0d3fac0e-bee0-4089-95c3-66fc203e801d)  verwenden, ohne die [FORCE-Prozessierungsumgebung](https://code-de.org/en/portfolio/?id=7e0e6df5-9d3d-40e9-a158-b0a203965445) selbst zu nutzen. Der FORCE-Datenwürfel wird hier ausführlich beschrieben: https://force-eo.readthedocs.io/en/latest/howto/datacube.html In der Filestruktur liegen die mit FORCE atmospärenkorrigten als BOA (Bottom of Atmosphere) Prodult bereits vor - ebenso die Wolkenmasken.

Die Daten mit dem Datenwürfel werden hier näher beschrieben: https://force-eo.readthedocs.io/en/latest/components/lower-level/level2/format.html

Bitte beachten: wir nutzen in diesem Beispiel den FORCE data cube der über den NFS-share zur Verfügung gestellt wird. In der EO-Lab / CODE-DE JupyterLab Umgebung ist dieser bereits als Verzeichnis gemountet. Mehr Informationen, wie Sie diesen in Ihrer eigenen Umgebung selbst mounten können, finden Sie [hier in unserer knowledgebase](https://knowledgebase.code-de.org/de/latest/eodata/How-to-mount-the-FORCE-community-collection-as-an-NFS-share-on-CODE-DE.html).  

Um die Daten in Python prozessieren zu können, benötigen wir einige standard Python Bibliotheken, die in der folgenden Zelle importiert werden.

In [ ]:
# Installieren der notwendigen Bibliotheken
import sys
!{sys.executable} -m pip install numpy 
!{sys.executable} -m pip install rasterio
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install pyproj

In [ ]:
# Importieren der notwendigen Bibliotheken
import rasterio
from rasterio import plot
import os
import numpy as np
import matplotlib.pyplot as plt
from rasterio import plot
from pyproj import Transformer

# Visualisieren aller Plots mit der selben Größe
plt.rcParams['figure.figsize'] = [12,12]

FORCE ist in einem eigenen Daten-System organisiert: in 30 km x 30 km Rasterdatenblöcken. Diese sogenannten "Tiles" sind in einer wohldefinierten Proketion: https://epsg.io/3035 abgelegt. Um zu wissen in welchem Tile wir uns befinden benötigen wir eine kleine Funktion, die uns das korrekte Tile ausrechnet (relativ zu einem bekannten Tile: hier X0044_Y0052).

Die beiden Variablen in der folgenden Zelle myX und myY rechnen die Dezimalkoordinaten unseres Standortes um und errechnen "unser" FORCE Tile. Jahr und Monat müssen ebenfalls angegeben werden. Damit ist das Wo und Wann definiert.


In [ ]:
# Koordinaten von Bonn: 7.1022/50.7258; Koordinaten von Essen: 7.014761/51.458069
myX = 7.014761
myY = 51.458069

# Definiere den Monat und das Jahr für die Untersuchung:
year='2020'
month='07'

tileSize = 30000

transformer = Transformer.from_crs("EPSG:4326", "EPSG:3035")
y1,x1=transformer.transform(49.992,2.3852)
y2,x2=transformer.transform(myY,myX)
print ("Linksobere Koordinate von X044 and Y052: ",x1,y1)
print ("Meine Koordinaten:", x2,y2)

In [ ]:
#x1,y1=reprojectPoint(4326,3035,2.3852,49.992) # upper left coordinate of X0044 und Y0052
#x2,y2=reprojectPoint(4326,3035,myX,myY) #Umprojektion von EPSG:4326 nach EPSG:3035

deltaX= round((x2-x1) / tileSize)
deltaY= round((y2-y1) /tileSize)
tileX='X00'+str(44+deltaX)
tileY='Y00'+str(52-deltaY)
tile=tileX+'_'+tileY
print ("FORCE Kachel:", tile)

In [ ]:
# Um in einen bstimmten Ordner der Force Daten zu schauen, brauchen wir einige Paramter: 
# tile-x, tile_y and the date
#tile='X0055_Y0047/'
#year='2020'
#month='07'

print ("Suchen in:", tile, "/", year, month)

path = '/home/jovyan/force/ard/'+tile+'/'  
objects = os.listdir(path)

print("Suchen nach Dateien im gegebenen Ordner: " + path)
files=[]

for obj in objects: #for obj in objects['Contents']:
    if f'{str(year)+str(month)}' in obj:
        if '_BOA.tif' in obj:
            #print(obj)
            files.append(obj)
        
print(files)         

Für die Datenprozessierung benötigen wir eine Wolkenmaske, so dass bewölkte Pixel ignoriert werden können.
Bei Fragen zur Wolkenmaskierung in FORCE könnte dieses Tool hilfreich sein: https://bit-flag-renderer.readthedocs.io/en/latest/index.html

In [ ]:
def maskCloud(inFile):
  
  src=rasterio.open(inFile)
  out_meta = src.meta
  QA=src.read() << 12
  mask = np.where(((QA > 2**12) ),1,0)
  
  return mask

Nun kommt der Kern des Programms. Wir iteriern über alle Sentinel-2 Szenen, berechnen jeweils den EVI (oder wahlweise NDVI), maskieren die Wolken und summieren die wolkenfreien Pixel. Am Ende erfolgt die Division durch die Anzahl der wolkenfreien Pixel zur Mittelwertbildung.

In [ ]:
# Definieren des Pfades zur ersten Datei aus der Liste
fLocal = path+os.path.basename(files[0])

# Lesen der ersten Datei, um die ihre Metadaten zu erfassen
src=rasterio.open(fLocal)
band8=src.read([8])
out_meta = src.meta
src.close() # schließen des Bandes

# Erstellen von numpy arrays der gleichen Dimension
vi_sum = np.zeros_like(band8, dtype=np.float32)
vi_count = np.zeros_like(band8, dtype=np.uint8)

count=0
for f in files: # Schleife, die über alle Dateien iteriert
    fLocal = path+os.path.basename(f)
   
    fLocalQA = fLocal.replace('BOA','QAI')
   
    src=rasterio.open(fLocal)
    NIR=src.read([8])/10000.0 # Skalierung in Force: Reflektanz * 10000
    RED=src.read([4])/10000.0
    BLUE = src.read([2])/10000.0
    src.close() 
    
    imgDate=fLocal.split("/")[-1][0:8]         
    EVI = 2.5*(NIR-RED)/((NIR+6*RED-7.5*BLUE)+1) 
    #Alternativ mit NDVI die beiden folgden Zeilen auskommentieren.. 
    #NDVI = (NIR-RED)/(NIR+RED)
    #EVI = NDVI
    EVI[(EVI >1) | (EVI < -1)] = np.nan
    mask = np.zeros_like(NIR, dtype=np.uint8)
    clouds = maskCloud(fLocalQA) 
    #nodata value in FORCE=-9999  
    mask = np.where((NIR == -9999) | np.isnan(EVI) | (clouds == 1), np.nan, 1)
    # Addition des EVI innerhalb der Schleife             
    vi_sum += np.where(np.isnan(mask), 0, EVI)
    vi_count += np.where(np.isnan(mask), 0, 1).astype(np.uint8)
    count += 1
    print("Prozessiertes Bild: ",count)
    src.close()

# Summe dividiert durch Anzahl ergibt den Mittelwert. Hier könnte z.B. der Median hilfreich sein.  
with np.errstate(invalid='ignore', divide='ignore'):
    vi_average = np.divide(vi_sum, vi_count, where=vi_count != 0)

# Optional: Konvertieren zu Integers, falls notwendig 
vi_average = np.nan_to_num(vi_average, nan=-9999).astype(np.float32)

print(f"Durchschnittlicher EVI berechnet für {count} Bilder.")

Zur Visualisuertung hier im Notebook nutzen wir eine simple Plot Funktion in pyplot: https://matplotlib.org/stable/tutorials/pyplot.html

In [ ]:
# RGB des letzen Bildes in der Liste
%matplotlib inline
RGB=np.stack([BLUE,NIR,RED],axis=-1).squeeze() 
RGB[(RGB > 1)] = 1
RGB[(RGB < 0)] = 0
plt.imshow(RGB)
plt.title(imgDate + ' - RGB: Blau, NIR, Rot')

Der gemittelte EVI über einen Monat als gemitteltes Bild-Komposit aller Aufnahmen im Monat.

In [ ]:
plt.imshow(np.squeeze(vi_average),cmap='jet',vmin=0,vmax=0.8)
plt.title('EVI')
plt.colorbar()

Die Anzahl der "guten" Pixel im Bild-Komposit als Resultat der Wolkenmaskierung.

In [ ]:
plt.imshow(np.squeeze(vi_count),cmap='inferno',vmin=0,vmax=count)
plt.title('Anzahl verwendeter Pixel')
plt.colorbar()

EVI und die Anzahl der "guten" Pixel werden nun als zwei seperate Dateien in das Verzeichnis geschrieben und können z.B: auf den lokalen Rechner heruntergeladen werden.

In [ ]:
# Schreiben des EVI Bildes
kwargs = out_meta
kwargs.update(
    dtype=rasterio.float32,count=1,
    compress='lzw')
with rasterio.open(year+month+'_evi.tif', 'w',**kwargs) as dst:
        dst.write(vi_average.astype(rasterio.float32))
        
# Schreiben des Bildes zur Anzahl der verwendeten Pixel
kwargs.update(
    dtype=rasterio.uint8,count=1,nodata=255)
with rasterio.open(year+month+'_count.tif', 'w',**kwargs) as dst:
        dst.write(vi_count.astype(rasterio.uint8))